In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


https://keras.io/applications/#inceptionv3

#### include_top
For ImageNet dataset, there are different 1000 labels to categorize the images. Thus, when you want to train a model with ImageNet dataset, you need to specify the number of neurons in the output (softmax) layer as 1000. However, we have such a dataset with ~50 -actually, 46- labels to categorize the images. We should not include the top (output, softmax, last, whatever you would like to call) layer of ResNet50 for our model, so we can add a new layer and specify the number of neurons as what the dataset needs.

https://towardsdatascience.com/transfer-learning-for-image-classification-using-keras-c47ccf09c8c8

In [23]:

img_width, img_height = 256, 256
input_shape = (1, img_width, img_height)
train_data_dir = "/root/finalproject/tensorflow-for-poets-2/tf_files/Training"
# validation_data_dir = "data/val"
# nb_train_samples = 4125
# nb_validation_samples = 466 
 
batch_size = 16
epochs = 100

In [3]:
base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape = None, pooling=None)
# create the base pre-trained model


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

https://towardsdatascience.com/https-medium-com-manishchablani-useful-keras-features-4bac0724734c

##### pooling: Optional pooling mode for feature extraction when include_top is False.  
None means that the output of the model will be the 4D tensor output of the last convolutional layer.  
'avg' means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor.  
'max' means that global max pooling will be applied.  

In [5]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- for this example we have 7 classes
predictions = Dense(7, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

In [7]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")


Found 28707 images belonging to 7 classes.


In [19]:
total_size = 28707 
steps_per_epoch = len(train_generator)
steps_per_epoch

1795

In [21]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inception_v3", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:

# Train the model 
model.fit_generator(
train_generator,
# samples_per_epoch = nb_train_samples,
epochs = epochs,
steps_per_epoch = steps_per_epoch,
# validation_data = validation_generator,
# nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

Epoch 1/100
1795/1795 [==============================] - 950s 529ms/step - loss: 1.7152 - acc: 0.3039

Epoch 00001: acc improved from -inf to 0.30376, saving model to inception_v3
Epoch 2/100
1795/1795 [==============================] - 950s 529ms/step - loss: 1.7091 - acc: 0.3057

Epoch 00002: acc improved from 0.30376 to 0.30567, saving model to inception_v3
Epoch 3/100
1795/1795 [==============================] - 949s 529ms/step - loss: 1.7059 - acc: 0.3101

Epoch 00003: acc improved from 0.30567 to 0.31020, saving model to inception_v3
Epoch 4/100
1795/1795 [==============================] - 948s 528ms/step - loss: 1.7053 - acc: 0.3151

Epoch 00004: acc improved from 0.31020 to 0.31480, saving model to inception_v3
Epoch 5/100
1795/1795 [==============================] - 951s 530ms/step - loss: 1.6946 - acc: 0.3209

Epoch 00005: acc improved from 0.31480 to 0.32107, saving model to inception_v3
Epoch 6/100
1795/1795 [==============================] - 953s 531ms/step - loss: 1.6950 

In [ ]:

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(16, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

# test_datagen = ImageDataGenerator(
# rescale = 1./255,
# horizontal_flip = True,
# fill_mode = "nearest",
# zoom_range = 0.3,
# width_shift_range = 0.3,
# height_shift_range=0.3,
# rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

In [ ]:
def layer_to_visualize(layer):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)

    print ('Shape of conv:', convolutions.shape)

    n = convolutions.shape[0]
    n = int(np.ceil(np.sqrt(n)))

    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(12,8))
    for i in range(len(convolutions)):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[i], cmap='gray')

# Specify the layer to want to visualize
layer_to_visualize(convout1)